In [ ]:
import sys
print(sys.executable)

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
from pathlib import Path
import os
import random
from plyfile import PlyData, PlyElement
import open3d as o3d
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
import tensorflow as tf
print("Versión de cuDNN:", tf.sysconfig.get_build_info()["cudnn_version"])
print("Versión de CUDA:", tf.sysconfig.get_build_info()["cuda_version"])

In [ ]:
import tensorflow as tf

print("Versión de TensorFlow:", tf.__version__)
print("GPUs disponibles:", tf.config.list_physical_devices('GPU'))

# GOOSE Dataset

In [8]:
import numpy as np
from pathlib import Path
from tqdm import tqdm

NUM_POINTS = 4096  # Puntos por fragmento

# Diccionario de mapeo: cada categoría tiene una lista de etiquetas originales.
category_mapping = {
    0: [43, 38, 58, 29, 41, 42, 44, 39, 55],  # Construction
    1: [4, 45, 6, 40, 60, 61, 33, 32, 14],      # Object
    2: [7, 22, 9, 26, 11, 21],                   # Road
    3: [48, 47, 1, 19, 46, 10, 25],              # Sign
    4: [23, 3, 24, 31, 2],                       # Terrain  
    5: [51, 50, 5, 18],                          # Drivable Vegetation
    6: [28, 27, 62, 52, 16, 30, 59, 17],          # Non Drivable Vegetation
    7: [13, 15, 12, 36, 57, 49, 20, 35, 37, 34, 63],  # Vehicle
    8: [8, 56, 0, 53, 54],                        # Void
}

# Generamos un diccionario inverso: etiqueta original -> categoría
label_to_category = {
    label: cat for cat, labels in category_mapping.items() for label in labels
}

def map_labels(labels: np.ndarray) -> np.ndarray:
    return np.array([label_to_category.get(label, 8) for label in labels], dtype=np.uint8)

def load_bin_and_label(bin_path: Path, label_path: Path):
    points = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4)[:, :3]
    labels = np.fromfile(label_path, dtype=np.uint32) & 0xFFFF
    labels = map_labels(labels)
    return points, labels

def process_file(bin_path: Path, label_path: Path):
    points, labels = load_bin_and_label(bin_path, label_path)
    num_points = points.shape[0]
    n_chunks = num_points // NUM_POINTS  # Número de fragmentos completos
    if n_chunks == 0:
        return None, None
    # Recortar y redimensionar
    points = points[:n_chunks * NUM_POINTS].reshape(n_chunks, NUM_POINTS, 3)
    labels = labels[:n_chunks * NUM_POINTS].reshape(n_chunks, NUM_POINTS)
    return points, labels

def process_path(x_path, y_path):
    x_files = sorted(os.listdir(x_path))
    y_files = sorted(os.listdir(y_path))

    x_data = []
    y_data = []

    for x, y in tqdm(zip(x_files, y_files), total=len(x_files), desc="Procesando archivos", unit="archivo"):
        points, labels = process_file(os.path.join(x_path, x), os.path.join(y_path, y))
        x_data.append(points)
        y_data.append(labels)

    return x_data, y_data

In [14]:

x_train_path = "/home/fmartinez/datasets/lidar/train"
y_train_path = "/home/fmartinez/datasets/labels/train"
x_val_path   = "/home/fmartinez/datasets_val/lidar/val"
y_val_path   = "/home/fmartinez/datasets_val/labels/val"

x_val, y_val = process_path(x_val_path, y_val_path)

Procesando archivos: 100%|██████████| 961/961 [05:28<00:00,  2.92archivo/s]


TypeError: cannot unpack non-iterable NoneType object

In [15]:
x_val

NameError: name 'x_val' is not defined

In [ ]:
indices_permutados_train = np.random.permutation(x_train.shape[0])
indices_permutados_val = np.random.permutation(x_val.shape[0])

x_train_shuffle = x_train[indices_permutados_train]
y_train_shuffle = y_train[indices_permutados_train]
x_val_shuffle = x_val[indices_permutados_val]
y_val_shuffle = y_val[indices_permutados_val]

x_train_shuffle.shape, y_train_shuffle.shape, x_val_shuffle.shape, y_val_shuffle.shape

In [ ]:
assert len(x_train) == len(y_train) and len(x_val) == len(y_val)
print(f"El conjunto de entrenamiento tiene {len(y_train)} nubes de puntos de {y_train.shape[0]} puntos")
print(f"El conjunto de entrenamiento tiene {len(y_val)} nubes de puntos de {y_val.shape[0]} puntos")

In [10]:
import plotly.graph_objects as go
import numpy as np

def plot_3D(xyz, labels):
    """
    Visualiza una nube de puntos en 3D con Plotly.
    - xyz: (num_points, 3) array con coordenadas (X, Y, Z).
    - labels: (num_points,) array con etiquetas semánticas.
    """

    # Definir 9 colores predefinidos en formato RGB
    predefined_colors = [
        "rgb(255, 0, 0)",    # Rojo
        "rgb(0, 255, 0)",    # Verde
        "rgb(0, 0, 255)",    # Azul
        "rgb(255, 255, 0)",  # Amarillo
        "rgb(255, 165, 0)",  # Naranja
        "rgb(128, 0, 128)",  # Púrpura
        "rgb(0, 255, 255)",  # Cian
        "rgb(255, 192, 203)",# Rosa
        "rgb(128, 128, 128)" # Gris
    ]

    # Asignar colores según la etiqueta (se asume que las etiquetas van de 0 a 8)
    point_colors = [predefined_colors[label % len(predefined_colors)] for label in labels]

    # Crear la figura 3D en Plotly
    fig = go.Figure()
    fig.add_trace(go.Scatter3d(
        x=xyz[:, 0], y=xyz[:, 1], z=xyz[:, 2],  # Coordenadas X, Y, Z
        mode='markers',
        marker=dict(size=1, color=point_colors, opacity=0.8)  # Color basado en etiquetas
    ))

    # Configurar etiquetas y título
    fig.update_layout(
        title="Nube de Puntos con Etiquetas Semánticas",
        scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z")
    )

    fig.show()

In [ ]:
# Llamar a la función con la primera nube de puntos
plot_3D(x_train_shuffle[1000], y_train_shuffle[1000])

In [ ]:
import numpy as np
import plotly.express as px

# Contar cuántos puntos hay por clase
unique_classes, class_counts = np.unique(y_train, return_counts=True)

# Mostrar distribución de clases
print(len(y_train))
for cls, count in zip(unique_classes, class_counts):
    print(f"Clase {cls}: {count} puntos")

# Crear gráfico interactivo con Plotly
fig = px.bar(x=unique_classes, y=class_counts, labels={'x': 'Clase', 'y': 'Cantidad de puntos'},
             title='Distribución de etiquetas en y_train')
fig.show()

In [ ]:
# Calcular pesos inversamente proporcionales a la frecuencia de cada clase
total_samples = len(y_train.flatten())  # Total de puntos
class_weights = {cls: total_samples / (len(unique_classes) * count) for cls, count in zip(unique_classes, class_counts)}

print("Pesos de las clases:", class_weights)

In [14]:
import tensorflow as tf
from tensorflow.keras import layers, Model

MAX_POINTS = 16384

def tnet(inputs, num_features):
    x = layers.Conv1D(64, 1, activation='relu', padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1D(128, 1, activation='relu', padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1D(1024, 1, activation='relu', padding="same")(x)
    x = layers.BatchNormalization()(x)

    x = layers.GlobalMaxPooling1D()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)

    x = layers.Dense(num_features * num_features, kernel_initializer='zeros',
                     bias_initializer=tf.keras.initializers.Constant(tf.eye(num_features).numpy().flatten()))(x)
    transform_matrix = layers.Reshape((num_features, num_features))(x)

    def transform(inputs_and_matrix):
        inputs, matrix = inputs_and_matrix
        return tf.matmul(inputs, matrix)

    transformed_inputs = layers.Lambda(transform)([inputs, transform_matrix])
    transformed_inputs = layers.Lambda(lambda t: tf.ensure_shape(t, (None, MAX_POINTS, num_features)))(transformed_inputs)

    return transformed_inputs

def build_pointnet(num_classes, input_dim=3, max_points=MAX_POINTS):
    inputs = tf.keras.Input(shape=(None, input_dim))

    x = tnet(inputs, input_dim)

    x = layers.Conv1D(64, 1, activation='relu', padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1D(128, 1, activation='relu', padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1D(64, 1, activation='relu', padding="same")(x)
    x = layers.BatchNormalization()(x)

    x = tnet(x, 64)

    x = layers.Conv1D(1024, 1, activation='relu', padding="same")(x)
    x = layers.BatchNormalization()(x)

    global_features = layers.GlobalMaxPooling1D()(x)
    global_features = layers.Lambda(lambda t: tf.expand_dims(t, axis=1))(global_features)
    global_features = layers.Lambda(lambda t: tf.repeat(t, repeats=max_points, axis=1))(global_features)

    x = layers.Lambda(lambda t: tf.ensure_shape(t, (None, max_points, 1024)))(x)

    x = layers.concatenate([x, global_features], axis=-1)

    x = layers.Conv1D(512, 1, activation='relu', padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1D(256, 1, activation='relu', padding="same")(x)
    x = layers.BatchNormalization()(x)

    outputs = layers.Conv1D(num_classes, 1, activation='softmax')(x)

    return Model(inputs, outputs)

In [15]:
import tensorflow as tf

class MeanIoUWrapper(tf.keras.metrics.Metric):
    def __init__(self, num_classes, name="mean_iou_wrapper", **kwargs):
        super(MeanIoUWrapper, self).__init__(name=name, **kwargs)
        self.num_classes = num_classes
        self.metric = tf.keras.metrics.MeanIoU(num_classes=num_classes)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred_labels = tf.argmax(y_pred, axis=-1)  # Convertir (batch, 16384, 9) -> (batch, 16384)
        self.metric.update_state(y_true, y_pred_labels)

    def result(self):
        return self.metric.result()

    def reset_state(self):
        self.metric.reset_state()

class MeanIoUWrapper_2(tf.keras.metrics.Metric):
    def __init__(self, num_classes, name="mean_iou_wrapper", **kwargs):
        super(MeanIoUWrapper, self).__init__(name=name, **kwargs)
        self.num_classes = num_classes
        self.metric = tf.keras.metrics.MeanIoU(num_classes=num_classes)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.int32)  # ✅ Asegurar que y_true sea int32
        y_pred_labels = tf.argmax(y_pred, axis=-1)  # Convertir (batch, 16384, 9) -> (batch, 16384)

        self.metric.update_state(y_true, y_pred_labels, sample_weight)  # ✅ Pasar sample_weight

    def result(self):
        return self.metric.result()

    def reset_state(self):
        self.metric.reset_state()


In [16]:
import tensorflow as tf

# Convertir los pesos a tensores
class_weight_tensor = tf.constant([class_weights[i] for i in range(len(unique_classes))], dtype=tf.float32)

def weighted_loss(y_true, y_pred):
    """Aplica pesos de clases a la pérdida de entropía cruzada"""
    y_true = tf.cast(y_true, tf.int32)
    sample_weights = tf.gather(class_weight_tensor, y_true)  # Asigna el peso según la clase
    loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
    return loss * sample_weights  # Escala la pérdida por el peso de la clase

def focal_loss(alpha=0.25, gamma=2.0):
    def loss(y_true, y_pred):
        # Convertir y_true a int32 por seguridad
        y_true = tf.cast(y_true, tf.int32)

        # Aplicar softmax si aún no se ha aplicado
        y_pred = tf.nn.softmax(y_pred, axis=-1)

        # Extraer las probabilidades correctas usando one-hot encoding
        y_true_one_hot = tf.one_hot(y_true, depth=tf.shape(y_pred)[-1])  # (batch, puntos, clases)
        y_pred = tf.reduce_sum(y_pred * y_true_one_hot, axis=-1)  # (batch, puntos)

        # Evitar log(0)
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1.0 - 1e-7)

        # Calcular la Focal Loss
        ce = -tf.math.log(y_pred)
        weight = alpha * tf.pow(1 - y_pred, gamma)
        
        return tf.reduce_mean(weight * ce)
    
    return loss

In [18]:
INPUT_DIM = 3
NUM_CLASSES = 9

# Definir el modelo PointNet
model = build_pointnet(num_classes=NUM_CLASSES, input_dim=INPUT_DIM)
mean_iou_wrapper = MeanIoUWrapper(num_classes=NUM_CLASSES)
optimizer = tf.optimizers.Adam(learning_rate=0.0003)

# Compilar el modelo
model.compile(
    optimizer = optimizer,
    loss=weighted_loss,
    metrics=["accuracy", mean_iou_wrapper]
)

In [ ]:
history = model.fit(
    x_train_shuffle,  
    y_train_shuffle,  
    validation_data=(x_val_shuffle, y_val_shuffle), 
    epochs=60,
    batch_size = 16,
    verbose=1
)

In [20]:
model.save("pointnet_goose_16k_3.keras")

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Supongamos que estos vienen de tu modelo entrenado (Keras history)
loss = history.history['loss']
val_loss = history.history.get('val_loss')
mean_iou = history.history.get('mean_iou_wrapper')
val_mean_iou = history.history.get('val_mean_iou_wrapper')
accuracy = history.history.get('accuracy')
val_accuracy = history.history.get('val_accuracy')

# Crear figura con 1 fila y 3 columnas de subplots
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=("Loss Curve", "Mean IoU Curve", "Accuracy Curve")
)

# 1) GRÁFICO DE PÉRDIDA (col=1)
epochs_loss = list(range(1, len(loss) + 1))

fig.add_trace(
    go.Scatter(
        x=epochs_loss,
        y=loss,
        mode='lines+markers',
        name='Train Loss'
    ),
    row=1, col=1
)

if val_loss:
    epochs_val_loss = list(range(1, len(val_loss) + 1))
    fig.add_trace(
        go.Scatter(
            x=epochs_val_loss,
            y=val_loss,
            mode='lines+markers',
            name='Validation Loss'
        ),
        row=1, col=1
    )

fig.update_xaxes(title_text='Epochs', row=1, col=1)
fig.update_yaxes(title_text='Loss', row=1, col=1, range=[0, 10])  # Rango ajustado

# 2) GRÁFICO DE Mean IoU (col=2) -> Limitar valores a [0,1]
if mean_iou:
    mean_iou = np.clip(mean_iou, 0, 1)
    epochs_mean_iou = list(range(1, len(mean_iou) + 1))
    fig.add_trace(
        go.Scatter(
            x=epochs_mean_iou,
            y=mean_iou,
            mode='lines+markers',
            name='Train Mean IoU'
        ),
        row=1, col=2
    )

    if val_mean_iou:
        val_mean_iou = np.clip(val_mean_iou, 0, 1)
        epochs_val_mean_iou = list(range(1, len(val_mean_iou) + 1))
        fig.add_trace(
            go.Scatter(
                x=epochs_val_mean_iou,
                y=val_mean_iou,
                mode='lines+markers',
                name='Validation Mean IoU'
            ),
            row=1, col=2
        )

fig.update_xaxes(title_text='Epochs', row=1, col=2)
fig.update_yaxes(title_text='Mean IoU', row=1, col=2, range=[0, 1])

# 3) GRÁFICO DE ACCURACY (col=3) -> Limitar valores a [0,1]
if accuracy:
    accuracy = np.clip(accuracy, 0, 1)
    epochs_acc = list(range(1, len(accuracy) + 1))
    fig.add_trace(
        go.Scatter(
            x=epochs_acc,
            y=accuracy,
            mode='lines+markers',
            name='Train Accuracy'
        ),
        row=1, col=3
    )

    if val_accuracy:
        val_accuracy = np.clip(val_accuracy, 0, 1)
        epochs_val_acc = list(range(1, len(val_accuracy) + 1))
        fig.add_trace(
            go.Scatter(
                x=epochs_val_acc,
                y=val_accuracy,
                mode='lines+markers',
                name='Validation Accuracy'
            ),
            row=1, col=3
        )

fig.update_xaxes(title_text='Epochs', row=1, col=3)
fig.update_yaxes(title_text='Accuracy', row=1, col=3, range=[0, 1])

# Ajustes generales de la figura
fig.update_layout(
    width=1300,
    height=500,
    showlegend=True
)

fig.show()